<div style="text-align:left; margin: 20px 0;">
  <img src="artwork/virtualbackground_langella.jpg" 
       alt="Seminario Estate GIS 2025 - Giuliano Langella" 
       style="max-width:50%; height:auto; border-radius:8px; box-shadow:0 2px 6px rgba(0,0,0,0.2);" />
</div>

# Confronto con modelli CNN pre-addestrati
## VGG16 / ResNet-50 / Inception
### Modelli pre-addestrati a confronto

| Modello       | Input richiesto (ImageNet) | Canali supportati | # Parametri (circa) | Note principali |
|---------------|----------------------------|-------------------|----------------------|-----------------|
| **VGG16**     | 224 × 224                  | 3 (RGB)           | ~138 M               | Architettura “classica”: sequenza di blocchi Conv+Pool → Dense. Molto pesante ma didattica. |
| **ResNet-50** | 224 × 224                  | 3 (RGB)           | ~25 M                | Introduce i *residual connections* → rete più profonda senza vanishing gradient. Ottimo compromesso accuratezza/costo. |
| **InceptionV3** | 299 × 299                | 3 (RGB)           | ~23 M                | Usa moduli *Inception* con kernel paralleli di diversa dimensione. Più complessa, input più grande. |

> **Nota:**  
> - Tutti i modelli sono pre-addestrati su ImageNet (3 canali RGB).  
> - Con dati multispettrali (13 bande) si può:  
>   1. Usare solo RGB (B4-B3-B2).  
>   2. Proiettare 13→3 canali con `Conv2D(1×1)`.  
>   3. (meno consigliato) Adattare il primo layer a 13 canali → perdi compatibilità con i pesi ImageNet.  

## STEP 0 — Parametri

In questa cella definiamo **tutti i parametri** che useremo nel notebook, preservando alcuni parametri utilizzati per l'addestamento del modello CNN custom per un confronto equo.<br>
**Obiettivo:** poter cambiare modello/strategie **senza toccare altro codice**.

### Note didattiche
- Useremo split già pronti (*train/val/test*) prodotti dal notebook baseline.  
- Sceglieremo un **backbone pre-addestrato** su ImageNet (*VGG16 / ResNet-50 / InceptionV3*).  
- Poiché i backbone ImageNet vogliono **3 canali (RGB)**, partiremo semplice:  
  - `CHANNEL_STRATEGY = "RGB_only"` → selezioniamo le bande Sentinel-2 **B4-B3-B2 (R-G-B)**.  
- Faremo training in **due stadi**:  
  1. **Stage 1**, <b style="color:magenta">feature extractor mode</b>: il backbone è congelato (weights non aggiornati), si addestra solo la testa (classification head).<br> **In parole semplici:** <b style="color:green">teniamo fissa la parte iniziale (backbone)</b> che ha già imparato tante caratteristiche utili sulle immagini (linee, forme, texture), e <b style="color:green">alleniamo solo la parte finale (head)</b>, che deve imparare a distinguere le nostre classi specifiche (es. “Forest”, “Residential”, ecc.).
  2. **Stage 2**, <b style="color:magenta">fine-tuning mode</b>: si sbloccano alcuni (o tutti) i blocchi convoluzionali e si allenano con un learning rate ridotto, così il modello affina le feature pre-esistenti al nostro dominio
  3.  sblocco (in parte o tutto) il backbone, con learning rate più basso (*fine-tuning*).

### Nota sul confronto tra modelli

Per confrontare correttamente modelli diversi (es. CNN custom, VGG16, ResNet-50, InceptionV3) è importante **fissare alcuni elementi comuni**, in modo che le differenze nei risultati siano attribuibili all’architettura e non a scelte casuali:

- **Split dei dati**: train/val/test devono essere identici → garantito dai file `split_files`.
- **Batch size**: usiamo lo stesso valore (es. 64), salvo limiti di memoria.
- **Numero di epoche / early stopping**: impostiamo lo stesso schema di training.
- **Data augmentation**: stessi tipi di trasformazioni su tutti i modelli.
- **Callback di training**: stesso set di strategie (`EarlyStopping`, `ReduceLROnPlateau`, ecc.).

Alcuni parametri possono cambiare (es. learning rate iniziale o freeze/unfreeze per modelli pre-addestrati), ma questi saranno **esplicitati** caso per caso.  
Così rendiamo il confronto “pulito": ciò che cambia è il **modello**, non il contesto dell’esperimento.

### Codice Python

In [1]:
import os
from pathlib import Path
from datetime import datetime

In [2]:
# ========== PERCORSI ==========
# Cartella che contiene i 3 CSV: train_list.csv, val_list.csv, test_list.csv
# (Generati dal notebook baseline; qui basta puntare alla cartella giusta.)
SPLIT_DIR = "outputs/20250908_102502"

# Radice dove salveremo i risultati di questo notebook
SAVE_ROOT = "outputs_pretrained"

# ========== MODELLO PRE-ADDESTRATO ==========
# Scegli il backbone da usare (scrivi in minuscolo come sotto):
#   "resnet50" | "vgg16" | "inception_v3"
MODEL_NAME = "resnet50"

# Dimensione input richiesta dal modello (H, W)
# - VGG16 / ResNet50: 224 x 224
# - InceptionV3:      299 x 299 (tipico)
if MODEL_NAME == "inception_v3":
    INPUT_SIZE = (299, 299)
else:
    INPUT_SIZE = (224, 224)

# ========== STRATEGIA CANALI (13 bande → 3 canali RGB) ==========
# Come adattiamo le 13 bande multispettrali ai pesi ImageNet (3 canali)?
# - "RGB_only"          : usa B4-B3-B2 (R-G-B). Semplice e immediato.
# - "1x1_conv_to_RGB"   : proiezione 13→3 con Conv1x1 (trainabile).
# - "first_conv_adapt"  : (sconsigliato) modifica primo layer del backbone a 13 canali.
CHANNEL_STRATEGY = "RGB_only"   # per il seminario partiamo da qui

# Indici RGB per Sentinel-2 in EuroSAT MS (0-based sugli array letti): B4=3 (R), B3=2 (G), B2=1 (B)
RGB_IDX = [3, 2, 1]

# ========== TRAINING A DUE STADI ==========
# Numero di epoche per ogni stadio
FREEZE_EPOCHS = 5    # Stage 1: solo testa
FT_EPOCHS     = 15   # Stage 2: fine-tuning (backbone sbloccato)

# Learning rate per ogni stadio
LR_HEAD = 1e-3   # testa (più alto)
LR_FT   = 1e-4   # fine-tuning (più basso)

In [3]:
import os, json

def load_common_hparams(split_dir, default_batch=64, default_augment=True, default_seed=42):
    """
    Legge iperparametri condivisi dal run baseline (cartella SPLIT_DIR).
    Cerca un file 'config.json' e, se presente, estrae BATCH_SIZE, AUGMENT, SEED.
    Se non trova il file o i campi, usa i default passati.

    Ritorna: (batch_size:int, augment:bool, seed:int, classes_or_None:list|None)
    """
    cfg_path = os.path.join(split_dir, "config.json")
    batch_size = default_batch
    augment    = default_augment
    seed       = default_seed
    classes    = None

    if os.path.isfile(cfg_path):
        try:
            with open(cfg_path, "r") as f:
                cfg = json.load(f)
            # campi attesi; usa default se assenti
            if "batch_size" in cfg:
                batch_size = int(cfg["batch_size"])
            if "seed" in cfg:
                seed = int(cfg["seed"])
            if "augment" in cfg:
                # accetta bool o 0/1
                augment = bool(cfg["augment"])
            if "classes" in cfg and isinstance(cfg["classes"], list):
                classes = cfg["classes"]
            print(f"[INFO] Parametri letti da {cfg_path}")
        except Exception as e:
            print(f"[WARN] Impossibile leggere {cfg_path}: {e}. Uso i default.")
    else:
        print(f"[WARN] File non trovato: {cfg_path}. Uso i default.")

    print(f"[INFO] BATCH_SIZE = {batch_size}")
    print(f"[INFO] AUGMENT    = {augment}")
    print(f"[INFO] SEED       = {seed}")    
    return batch_size, augment, seed, classes

In [4]:
# ========== LETTURA IPERPARAMETRI COMUNI ==========
BATCH_SIZE, AUGMENT, SEED, CLASSES = load_common_hparams(SPLIT_DIR)

# Se vogliamo stampare anche le classi (se presenti nel config.json)
if CLASSES is not None:
    print(f"[INFO] Classi dal baseline: {CLASSES}")

[INFO] Parametri letti da outputs/20250908_102502/config.json
[INFO] BATCH_SIZE = 64
[INFO] AUGMENT    = True
[INFO] SEED       = 3008
[INFO] Classi dal baseline: ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


In [5]:
# ========== CARTELLA DI SALVATAGGIO (con timestamp) ==========
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
RUN_DIR = Path(SAVE_ROOT) / f"{timestamp}_{MODEL_NAME}"
os.makedirs(RUN_DIR, exist_ok=True)

# ========== STAMPA RIEPILOGO ==========
print("=== CONFIG PRE-TRAINED NOTEBOOK ===")
print(f"SPLIT_DIR        : {Path(SPLIT_DIR).resolve()}")
print(f"MODEL_NAME       : {MODEL_NAME}")
print(f"INPUT_SIZE (HxW) : {INPUT_SIZE}")
print(f"CHANNEL_STRATEGY : {CHANNEL_STRATEGY}  (RGB_IDX={RGB_IDX} se RGB_only)")
print(f"FREEZE_EPOCHS    : {FREEZE_EPOCHS}  |  FT_EPOCHS : {FT_EPOCHS}")
print(f"LR_HEAD          : {LR_HEAD}      |  LR_FT     : {LR_FT}")
print(f"BATCH_SIZE       : {BATCH_SIZE}")
print(f"AUGMENT          : {AUGMENT}")
print(f"SEED             : {SEED}")
print(f"RUN_DIR          : {RUN_DIR.resolve()}")

=== CONFIG PRE-TRAINED NOTEBOOK ===
SPLIT_DIR        : /home/jovyan/work/outputs/20250908_102502
MODEL_NAME       : resnet50
INPUT_SIZE (HxW) : (224, 224)
CHANNEL_STRATEGY : RGB_only  (RGB_IDX=[3, 2, 1] se RGB_only)
FREEZE_EPOCHS    : 5  |  FT_EPOCHS : 15
LR_HEAD          : 0.001      |  LR_FT     : 0.0001
BATCH_SIZE       : 64
AUGMENT          : True
SEED             : 3008
RUN_DIR          : /home/jovyan/work/outputs_pretrained/20250909_195018_resnet50


## STEP 1 — Riutilizzo di split e classi

In questo step:
- Carichiamo i tre file CSV (`train_list.csv`, `val_list.csv`, `test_list.csv`) dal `SPLIT_DIR`.
- Ogni riga del CSV contiene: percorso del file, etichetta testuale.
- Ricaviamo la lista delle classi (`class_names`) mantenendo coerenza con il run baseline (se leggiamo `CLASSES` dal `config.json`).
- Creiamo i dizionari di mapping `label2id` e `id2label`.
- Stampiamo un riepilogo dei conteggi totali e per classe in ciascuno split.

In [6]:
import os
import csv
from collections import Counter

Definizione di funzioni:

In [7]:
# Funzione di utilità per leggere una lista da CSV
def read_split_list(csv_path):
    with open(csv_path, "r", newline="") as f:
        reader = csv.reader(f)
        next(reader, None)  # salta l'intestazione
        data = []
        for row in reader:
            if not row:
                continue
            filepath, label = row[0], row[1]
            data.append((filepath, label))
    return data

# Funzione di conteggio per split
def count_per_class(data):
    labels = [label for _, label in data]
    return Counter(labels)

#### 1) Carica i tre split

In [8]:
train_list = read_split_list(os.path.join(SPLIT_DIR, "train_list.csv"))
val_list   = read_split_list(os.path.join(SPLIT_DIR, "val_list.csv"))
test_list  = read_split_list(os.path.join(SPLIT_DIR, "test_list.csv"))

print(f"[INFO] Letture completate: train={len(train_list)}, val={len(val_list)}, test={len(test_list)}")

[INFO] Letture completate: train=4000, val=1000, test=2000


#### 2) Classi: se definite nel config baseline, usa quelle

In [9]:
if "CLASSES" in globals() and CLASSES:
    class_names = CLASSES
else:
    # fallback: ricava da train_list
    class_names = sorted(list({label for _, label in train_list}))

label2id = {label: idx for idx, label in enumerate(class_names)}
id2label = {idx: label for label, idx in label2id.items()}

print(f"[INFO] Classi ({len(class_names)}): {class_names}")

[INFO] Classi (10): ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


#### 3) Conteggi per split

In [10]:
for split_name, data in [("TRAIN", train_list), ("VAL", val_list), ("TEST", test_list)]:
    counter = count_per_class(data)
    print(f"\n=== {split_name} ===")
    print(f"Totale: {len(data)}")
    for cls in class_names:
        print(f"{cls:20s}: {counter.get(cls, 0)}")


=== TRAIN ===
Totale: 4000
AnnualCrop          : 400
Forest              : 400
HerbaceousVegetation: 400
Highway             : 400
Industrial          : 400
Pasture             : 400
PermanentCrop       : 400
Residential         : 400
River               : 400
SeaLake             : 400

=== VAL ===
Totale: 1000
AnnualCrop          : 100
Forest              : 100
HerbaceousVegetation: 100
Highway             : 100
Industrial          : 100
Pasture             : 100
PermanentCrop       : 100
Residential         : 100
River               : 100
SeaLake             : 100

=== TEST ===
Totale: 2000
AnnualCrop          : 200
Forest              : 200
HerbaceousVegetation: 200
Highway             : 200
Industrial          : 200
Pasture             : 200
PermanentCrop       : 200
Residential         : 200
River               : 200
SeaLake             : 200


## STEP 2 — Pipeline dati (tf.data) con strategia `RGB_only`

In questo step costruiamo le pipeline di dati per i modelli **pre-addestrati su ImageNet**.  
Poiché questi backbone sono stati addestrati solo su immagini **RGB**, riduciamo le patch EuroSAT da **13 bande multispettrali** a **3 bande (B4–B3–B2 → R–G–B)**.

 > Le funzioni utilizzate nella procedura per l'addestramento del modell CNN custom sono state raccolte nel file `data_pipeline_pretrained.py`. 
Ulteriori funzioni sono state create ed aggiunte nel file per consentire la preparazione dei dati per i modelli pre-addestrati.
Una documentazione esaustiva è disponibile nel file `data_pipeline_pretrained.md`.

**Cosa facciamo concretamente:**
- Selezioniamo le bande B4, B3, B2 dall’immagine multispettrale.
- Ridimensioniamo ogni patch alla dimensione richiesta dal modello (224×224 per VGG16/ResNet50; 299×299 per InceptionV3).
- Applichiamo la funzione `preprocess_input` (vd data_pipeline_pretrained.py) specifica del backbone scelto:
  - VGG16/ResNet50 → conversione in BGR + sottrazione della media di ImageNet per canale.
  - InceptionV3 → scaling dei pixel nell’intervallo [-1,+1].
- Raggruppiamo in **batch** e abilitiamo il **prefetch**, così TensorFlow prepara i dati mentre il modello addestra.
- Aggiungiamo **augmentation semplice** (flip orizzontale) solo al train per migliorare la generalizzazione.

In questo modo, i dati che entrano nel backbone hanno **lo stesso formato e la stessa distribuzione statistica** delle immagini con cui i pesi sono stati originariamente addestrati su ImageNet.

### Codice Python

In [11]:
from data_pipeline_pretrained import make_dataset_pretrained

# Esempio per strategy = "RGB_only" (consigliata per partire)
train_ds = make_dataset_pretrained(
    file_list=train_list,
    label2id=label2id,
    batch_size=BATCH_SIZE,
    augment=AUGMENT,                    # True, o comunque identico al modello CNN custom
    seed=SEED,
    input_size=INPUT_SIZE,
    channel_strategy=CHANNEL_STRATEGY,  # "RGB_only"
    model_name=MODEL_NAME,              # es. "resnet50"
    rgb_idx=RGB_IDX                     # [3,2,1] → B4,B3,B2
)

val_ds = make_dataset_pretrained(
    file_list=val_list,
    label2id=label2id,
    batch_size=BATCH_SIZE,
    augment=False,            # niente augmentation su val/test
    seed=SEED,
    input_size=INPUT_SIZE,
    channel_strategy=CHANNEL_STRATEGY,
    model_name=MODEL_NAME,
    rgb_idx=RGB_IDX
)

test_ds = make_dataset_pretrained(
    file_list=test_list,
    label2id=label2id,
    batch_size=BATCH_SIZE,
    augment=False,
    seed=SEED,
    input_size=INPUT_SIZE,
    channel_strategy=CHANNEL_STRATEGY,
    model_name=MODEL_NAME,
    rgb_idx=RGB_IDX
)

## STEP 3 — Scelta & caricamento <b style="color:magenta">backbone_model</b>

In questo step scegliamo e carichiamo il **modello pre-addestrato** che useremo come *backbone* (estrattore di feature) (<b style="color:black">backbone_model</b>).

### Requisiti di input
- **VGG16 / ResNet50** → input atteso **224×224×3** (minimo 32×32, ma si usa sempre 224 per coerenza).  
- **InceptionV3** → input tipico **299×299×3** (minimo supportato 75×75).

### Modello scelto: ResNet-50
ResNet-50 è una CNN (Convolutional Neural Network), ma con una particolarità che l’ha resa una pietra miliare: è una Residual Network con 50 layer profondi (da qui il nome).
ResNet-50 rappresenta un passaggio evolutivo dalle CNN tradizionali (tipo VGG) verso architetture più stabili e scalabili.

#### Struttura di ResNet-50

La figura seguente mostra la struttura semplificata della **ResNet-50**:

<img src="artwork/ResNet50.png" alt="ResNet-50 architecture" width="800"/>

**Componenti principali:**
- **Zero Padding**: aggiunge margini artificiali per preservare la dimensione spaziale dopo la convoluzione iniziale.
- **Conv + BatchNorm + ReLU + MaxPool**: primo stadio che riduce la risoluzione e prepara le feature map.
- **Conv Block**: blocco con convoluzioni che cambia dimensioni/canali (stride o proiezione).
- **ID Block (Identity Block)**: blocco residuo che mantiene la stessa dimensione e aggiunge la connessione diretta (skip connection).
- **Stack di Conv/ID Blocks**: compongono la parte centrale della rete, dove si estraggono feature sempre più complesse.
- **Avg Pool + Flattening + FC (Fully Connected)**: riduzione finale e classificazione (per ImageNet → 1000 classi).

Il punto di forza di ResNet-50 sono le **skip connections**, che permettono di allenare reti molto più profonde senza incorrere nel problema del *vanishing gradient*. 

<div style="font-family:Chalkboard; font-size:14px; color:#1f77b4;">

##### Note alla struttura  

1) Stage 1 di ResNet-50 ha la stessa sequenza logica del blocco 1 della CNN custom (Conv → BN → ReLU → Pool), ma usa un filtro più grande e stride per ridurre subito la dimensione spaziale:<br>  
  - <b>ResNet-50</b>: Conv 7×7, stride 2 → riduzione più aggressiva.<br>  
  - <b>CNN custom</b>: Conv 3×3, stride 1 → riduzione più dolce.  

2) Stage 2–5 sono composti da più blocchi residui (<i>Conv Block</i> + <i>Identity Block</i>).<br>  
   - <b>Conv Block</b>: introduce nuove feature maps e può ridurre la **risoluzione della griglia HxW**, aumentando al contempo la profondità (canali).<br>  
   - <b>Identity Block</b>: mantiene la dimensione ma arricchisce la rappresentazione, sfruttando le connessioni residue (<i>skip connections</i>).<br>
   - L’alternanza di questi blocchi permette di allenare reti molto profonde senza perdere informazione o incorrere in <i>vanishing gradient</i>.<br>  
   - Struttura tipica: Stage 2 (3 blocchi), Stage 3 (4 blocchi), Stage 4 (6 blocchi), Stage 5 (3 blocchi).  

3) <b>Skip connections</b>: permettono al segnale in ingresso di bypassare un blocco convoluzionale e di essere sommato al suo output. In questo modo il blocco non deve imparare da zero una funzione complessa, ma solo una “correzione” rispetto all’identità.<br>
   - Immaginiamo un tubo con diversi filtri in sequenza: a volte l’acqua (l’informazione) passa da tutti i filtri, altre volte si crea un piccolo bypass che la fa arrivare subito in fondo.<br>  
   - Questo meccanismo evita che le informazioni importanti si perdano man mano che la rete diventa più profonda.<br>  
   - In pratica, le <b>skip connections</b> permettono di <u>allenare reti molto più profonde</u> mantenendo stabili i gradienti e combinando sia le trasformazioni complesse dei layer intermedi che la “memoria” dell’informazione originale.  

    L’intuizione che ha portato alle skip connections (He et al., 2015): quando i ricercatori hanno provato ad addestrare reti molto profonde (50, 101, 152 layer…), si sono accorti che:
      - aumentando la profondità, l’errore di training non scendeva (addirittura peggiorava rispetto a reti più superficiali!).
      - il problema non era l’overfitting, ma la difficoltà ad allenare la rete (vanishing/exploding gradient, blocchi che “appiattiscono” l’informazione).

    <b>L’idea di base è stata:</b> se un blocco non sa ancora fare meglio di “identità” (cioè passare l’input invariato), lasciamogli questa strada semplice a disposizione. In altre parole, è più facile imparare una “piccola correzione” all’input (F(x)) che ricostruire da zero tutta la trasformazione desiderata.<br>
    Il blocco convoluzionale calcola $F(x)$, lo skip passa direttamente $x$, e alla fine $y = F(x) + x$ (operazione eseguita ad esempio dallo strato `conv2_block1_add`).

Ogni blocco (Conv o Identity) elabora le mappe di attivazione per estrarre nuove feature o raffinarne di vecchie. Impilando tanti blocchi uno dopo l’altro, la rete costruisce una gerarchia di feature:
 - <b>Blocchi iniziali</b> → catturano pattern molto semplici (bordi, contrasti, forme elementari).
 - <b>Blocchi intermedi</b> → combinano i pattern di basso livello in strutture più complesse (texture, contorni di oggetti, forme ricorrenti).
 - <b>Blocchi finali</b> → riconoscono concetti molto astratti e specifici (per ImageNet: “orecchie di cane”, “volante di auto”; per noi: pattern tipici di “forest” vs “industrial”).
 - Lo <b>skip connection</b> evita che la rete dimentichi le feature utili dei blocchi precedenti: è come se dicesse “non butto via ciò che ho già capito, ma lo arricchisco con nuove trasformazioni”.

Quindi: più blocchi => più profondità concettuale, cioè la capacità di estrarre feature via via più astratte e rappresentative.
</div>

### Cosa facciamo concretamente
- Usiamo i modelli disponibili in `keras.applications`, caricati con:
  - `weights="imagenet"` → pesi addestrati su ImageNet.  
  - `include_top=False` → <b style="color:orange">escludiamo la parte finale di classificazione originale</b> (1000 classi ImageNet).
- In questo modo il backbone funge da **estrattore di feature generali** (bordi, texture, pattern).  
- Congeliamo tutti i pesi del backbone per lo [**Stage 1**](#stage_1_tr):
  - in questa fase addestriamo **solo la nuova testa** (il classificatore specifico per le 10 classi EuroSAT).  
  - il backbone resta “fisso”, così sfruttiamo al massimo le feature già imparate da ImageNet.  

Successivamente, nello [**Stage 2**](#stage_2_tr) sbloccheremo (in parte o del tutto) il backbone per il fine-tuning, con learning rate più basso.

<div style="font-family:'Helvetica Neue', Arial, sans-serif; font-size:13px; color:#555; line-height:1.5; background-color:#eeeeee; padding:10px; border-radius:6px;">
  <p><strong>C’è una distinzione da fare:</strong></p>
  <ul>
    <li>
      <strong>ImageNet</strong> come dataset completo (<em>ILSVRC/Full</em>): contiene circa
      14 milioni di immagini distribuite su quasi 22.000 classi (<em>WordNet synsets</em>).
    </li>
    <li>
      <strong>ImageNet Large Scale Visual Recognition Challenge (ILSVRC)</strong>:
      il sottoinsieme più usato per addestrare e confrontare i modelli contiene
      1.2 milioni di immagini su 1000 classi.
    </li>
  </ul>

  <p>
    Quando in Keras carichiamo un modello pre-addestrato con <code>weights="imagenet"</code>,
    i pesi sono quelli addestrati sulle <strong>1000 classi di ILSVRC</strong>, non sulle
    22.000 del dataset completo.
  </p>
</div>

### Codice Python

In [12]:
from tensorflow.keras import applications, layers, models

# 1. Selezione dinamica del backbone
if MODEL_NAME == "resnet50":   # Se il parametro scelto è "resnet50", carichiamo quel backbone
    backbone_model = applications.ResNet50(   # Crea un modello ResNet-50 pre-addestrato su ImageNet
        weights="imagenet",                   # Usa i pesi già addestrati su ILSVRC (1000 classi)
        include_top=False,                    # Esclude la testa originale (classificazione su 1000 classi)
        input_shape=INPUT_SIZE + (3,)         # Specifica la dimensione dell’input: (H, W, 3) → RGB, potevamo ometterla avendo rispettato il requisiti del modello
    )    
elif MODEL_NAME == "vgg16":
    backbone_model = applications.VGG16(
        weights="imagenet",
        include_top=False,
        input_shape=INPUT_SIZE + (3,)
    )
elif MODEL_NAME == "inception_v3":
    backbone_model = applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        input_shape=INPUT_SIZE + (3,)
    )
else:
    raise ValueError(f"Backbone {MODEL_NAME} non supportato")

# 2. Congeliamo il backbone per lo Stage 1 (feature extractor)
backbone_model.trainable = False

print(f"[INFO] Backbone caricato: {MODEL_NAME}, input={INPUT_SIZE + (3,)}")
print(f"[INFO] Parametri totali: {backbone_model.count_params():,}")

[INFO] Backbone caricato: resnet50, input=(224, 224, 3)
[INFO] Parametri totali: 23,587,712


In [13]:
backbone_model.summary()

Model: "resnet50"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

<a id="step4"></a>
## STEP 4 — Progettiamo la <b style="color:magenta">head_model</b> + modello finale
<h3 style="color:#aaa; font-weight:600; margin:0.4em 0;">(ResNet-50 + RGB_only, tralasciamo il resto per semplicità)</h3>

### Obiettivo della head
Prendere le feature estratte dal `backbone_model` (ResNet-50 senza top) e trasformarle in **probabilità sulle 10 classi EuroSAT**, con pochi parametri e buon controllo dell’overfitting. La progettiamo in modo **semplice** e coerente con lo **stage di training in due fasi**.

### Spiegazione

**Idea chiave**<br>
- **`backbone_model`** = ResNet-50 senza testa (include_top=False), **congelata**: estrae feature.  
- **`head_model`** = piccola rete che trasforma le feature in **probabilità** sulle 10 classi EuroSAT.  
- In **Stage 1** alleniamo **solo la head**, lasciando il backbone **congelato** (pesi fissi), così sfruttiamo le feature generali apprese su ImageNet e adaptiamo solo la parte finale al nostro dominio.

> Nota: qui assumiamo `CHANNEL_STRATEGY = "RGB_only"` e input a **3 canali** (B4-B3-B2) già ridimensionati a `INPUT_SIZE`.

**Cosa inseriamo nella head**  
 - **GlobalAveragePooling2D**: prende in input l’uscita del `backbone_model`.  
   - Con immagini **224×224×3** in ingresso, ResNet-50 produce una feature map di **7×7×2048**.  
   - La `GlobalAveragePooling2D` fa la media sui 7×7 → otteniamo un **vettore di 2048 valori** (uno per canale).

 - **Dense(128) + ReLU + Dropout(0.3)**: lo strato decisionale riduce i **2048 valori** a **128**, combinandoli in modo non lineare e regolarizzando con dropout.

 - **Dense(num_classes) + Softmax**: lo strato finale trasforma i **128 valori** in un vettore di dimensione **num_classes=10** (per EuroSAT), che rappresenta le probabilità di appartenenza alle classi.

<div style="font-family:'Helvetica Neue', Arial, sans-serif; font-size:12px; color:#555; line-height:1.5; background-color:#eeeeee; padding:10px; border-radius:6px;">
  <p><strong>Approfondimento sugli strati della head</strong></p>
  <ul>
    <li><b>Dense(128)</b>: layer completamente connesso, cioè un piccolo strato fully-connected tipico delle FFNN. Riceve i <b>2048 valori</b> in uscita dal backbone e li combina linearmente per produrre <b>128 nuove feature</b>. È come costruire un mix “intelligente” di tutte le feature estratte.</li>
    <li><b>ReLU</b>: funzione di attivazione che mantiene i valori positivi e azzera i negativi, introducendo non linearità. Permette al modello di rappresentare relazioni complesse.</li>
    <li><b>Dropout(0.3)</b>: durante il training spegne casualmente il 30% dei neuroni, obbligando la rete a non dipendere da pochi segnali. Funziona come una “regolarizzazione” per ridurre l’overfitting.</li>
    <li><b>Dense(num_classes) + Softmax</b>: ultimo layer. Trasforma i 128 valori in <b>10 numeri</b>, uno per classe. La Softmax converte questi numeri in <b>probabilità</b> (tutti tra 0 e 1 e somma = 1), così possiamo interpretare l’output come: “classe più probabile = …”.</li>
  </ul>
</div>

<div style="font-family:'Helvetica Neue', Arial, sans-serif; font-size:11px; color:#777; line-height:1.5; background-color:#f7f7f7; padding:10px; border-radius:6px;">

<b>Maggiori dettagli:</b>
  <h4>a) GlobalAveragePooling2D → perché prima una media spaziale?</h4>
  <ul>
    <li><b>Cosa fa:</b> converte la feature map finale del backbone da <b>7×7×2048</b> in un <b>vettore di 2048 valori</b> facendo la media su ogni canale.</li>
    <li><b>Perché serve:</b>
      <ul>
        <li><b>Riduce i parametri</b> drasticamente rispetto a Flatten (da ~12,8M a ~262k).</li>
        <li><b>Invarianza spaziale</b>: punta sul “quanto” sono presenti le feature, non “dove”.</li>
        <li><b>Coerenza con ImageNet</b>: molti backbone sono pensati per concludere con GAP.</li>
      </ul>
    </li>
  </ul>

  <h4>b) Dense(128) + ReLU → perché un piccolo collo di bottiglia?</h4>
  <ul>
    <li><b>Cosa fa:</b> prende i 2048 valori e li ricombina in <b>128 nuove feature</b>.</li>
    <li><b>Perché 128:</b> compromesso tra capacità e rischio overfitting.</li>
    <li><b>Perché ReLU:</b> introduce non linearità semplice e robusta.</li>
  </ul>

  <h4>c) Dropout(0.3) → perché una “goccia” di regolarizzazione?</h4>
  <ul>
    <li><b>Cosa fa:</b> spegne casualmente il 30% dei neuroni in training.</li>
    <li><b>Perché serve:</b> obbliga la rete a non dipendere da pochi neuroni forti, riducendo overfitting.</li>
  </ul>

  <h4>d) Dense(num_classes) + Softmax → perché così in uscita?</h4>
  <ul>
    <li><b>Cosa fa:</b> trasforma i 128 valori in <b>10 logit</b>, poi li converte in probabilità (somma=1).</li>
    <li><b>Perché Softmax:</b> consente l’uso di cross-entropy e interpretazione diretta come distribuzione.</li>
  </ul>

  <h4>e) Scelte alternative (quando e perché)</h4>
  <ul>
    <li><b>Più capacità:</b> più neuroni (es. 256) o un secondo Dense+ReLU.</li>
    <li><b>Più regolarizzazione:</b> dropout più alto o weight decay.</li>
    <li><b>GlobalMaxPooling2D:</b> se le feature salienti sono puntuali.</li>
  </ul>

  <h4>f) Nota su strategie canali</h4>
  <ul>
    <li><b>RGB_only:</b> input a 3 canali coerenti con i pesi ImageNet.</li>
    <li><b>13→3 con Conv1×1:</b> proiezione trainabile prima del backbone; la head resta identica.</li>
  </ul>

</div>

### Codice Python

#### Premessa

Assunzioni già definite nello STEP 0/1/2: MODEL_NAME, INPUT_SIZE, CHANNEL_STRATEGY e class_names:

In [31]:
print("=== CONFIG PRE-TRAINED NOTEBOOK ===")
print(f"MODEL_NAME       : {MODEL_NAME}")
print(f"INPUT_SIZE (HxW) : {INPUT_SIZE}")
print(f"CHANNEL_STRATEGY : {CHANNEL_STRATEGY}  (RGB_IDX={RGB_IDX} se RGB_only)")
print(f"[INFO] Classi ({len(class_names)}): {class_names}")

=== CONFIG PRE-TRAINED NOTEBOOK ===
MODEL_NAME       : resnet50
INPUT_SIZE (HxW) : (224, 224)
CHANNEL_STRATEGY : RGB_only  (RGB_IDX=[3, 2, 1] se RGB_only)
[INFO] Classi (10): ['AnnualCrop', 'Forest', 'HerbaceousVegetation', 'Highway', 'Industrial', 'Pasture', 'PermanentCrop', 'Residential', 'River', 'SeaLake']


Nello STEP 3 il backbone è caricato con `include_top=False` e `backbone.trainable = False`.

#### 0) Import

In [32]:
from tensorflow import keras
from tensorflow.keras import layers, Model

In [33]:
num_classes = len(class_names)

#### 1) Input RGB (coerente con lo STEP 3: ResNet-50 si aspetta 3 canali)

Ispezione input layer del backbone

In [34]:
inputs = keras.Input(shape=INPUT_SIZE + (3,), name="image_rgb")

#### 2) Passo nel backbone congelato (definito nello STEP 3 come `backbone_model`)

In [35]:
# qui "agganciamo" davvero l'input al backbone
features = backbone_model(inputs, training=False)    # es: (None, 7, 7, 2048)

**features** è un tensore simbolico che rappresenta l’uscita del backbone.

#### 3) Definisco la HEAD come piccolo modello riutilizzabile

In [36]:
head_input = keras.Input(shape=backbone_model.output_shape[1:], name="features_in")
x = layers.GlobalAveragePooling2D(name="gap")(head_input)
x = layers.Dense(128, activation="relu", name="head_dense")(x)
x = layers.Dropout(0.3, name="head_dropout")(x)
head_output = layers.Dense(num_classes, activation="softmax", name="pred")(x)
head_model = Model(head_input, head_output, name="head_model")

#### 4) Composizione finale: backbone (freeze) + head (trainable)

In [47]:
outputs = head_model(features)
model = Model(inputs, outputs, name="resnet50_rgb_full")

**outputs** è un tensore simbolico di probabilità, di forma (None, 10) nel caso EuroSAT.

#### 5) Verifiche rapide

In [48]:
print(f"[INFO] backbone_model.trainable = {backbone_model.trainable}")  # atteso: False
print(f"[INFO] head_model.trainable     = {head_model.trainable}")      # atteso: True

[INFO] backbone_model.trainable = False
[INFO] head_model.trainable     = True


In [49]:
model.summary()

Model: "resnet50_rgb_full"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ image_rgb (InputLayer)          │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ head_model (Functional)         │ (None, 10)             │       263,562 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,851,274 (90.99 MB)

 Trainable params: 15,239,562 (58.13 MB)

 Non-trainable params: 8,611,712 (32.85 MB)

<div style="background:#f2f2f2; color:#333333; font-size:12px; line-height:1.5; padding:10px 12px; border-radius:8px;">
  <p><strong>Note:</strong></p>
  <ul style="margin:3px 0 0 6px;">
    <li>
      <strong>Catena delle forme (sanity check veloce)</strong><br>
      <code>image_rgb (None, 224, 224, 3) → resnet50 (None, 7, 7, 2048) → head_model (None, 10)</code>.<br>
      <em>None</em> è la dimensione batch variabile; <strong>10</strong> è il numero di classi EuroSAT: se qui non è 10, abbiamo cablato male la head.
    </li>
    <li>
      <strong>Backbone congelato davvero</strong><br>
      <code>resnet50 (Functional)</code> mostra <strong>23,587,712</strong> parametri tutti in <em>Non-trainable</em>. È la prova che lo Stage-1 allena solo la head.
    </li>
    <li>
      <strong>Costo della head (quanto “paga” aggiungere la nostra testa)</strong><br>
      <code>head_model (Functional)</code> ha <strong>263,562</strong> parametri <em>trainable</em> (~1&nbsp;MB): è molto più leggera del backbone. La leggerezza della head riduce sia il rischio di overfitting su dataset piccoli, sia i tempi di addestramento su dataset grandi.
    </li>
    <li>
      <strong>Perché 7×7×2048?</strong><br>
      ResNet-50 con <code>include_top=False</code> e input <code>224×224</code> produce l’ultima feature map di <strong>7×7</strong> con <strong>2048</strong> canali. La <code>GlobalAveragePooling2D</code> la comprime in un vettore di <strong>2048</strong> numeri, su cui la head costruisce la decisione.
    </li>
  </ul>
</div>

<a id="step5"></a>
## STEP 5 — Training

### Codice Python

<a id="stage_1_tr"></a>
#### 1) STAGE 1 — alleno SOLO la head (backbone congelato)

##### Callback – HEAD

In [50]:
early_stop = keras.callbacks.EarlyStopping(
    monitor="val_loss", 
    patience=5, 
    restore_best_weights=True
)
reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.5, 
    patience=2, 
    min_lr=1e-6
)

In [51]:
ckpt_path = RUN_DIR / "best_weights_stage1.keras"   # poi useremo un file diverso per lo stage 2
ckpt = keras.callbacks.ModelCheckpoint(
    filepath=str(ckpt_path),
    monitor="val_loss", 
    save_best_only=True
)
callbacks = [early_stop, reduce_lr, ckpt]

In [52]:
backbone_model.trainable = False        # sicurezza: resta congelato
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR_HEAD),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
print("\n[STAGE 1] Train only head")
history_s1 = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=FREEZE_EPOCHS, 
    callbacks=callbacks, 
    verbose=1
)
# salvo i pesi migliori dello stage 1 (già gestiti da ModelCheckpoint)
print(f"[INFO] Best weights (stage1): {ckpt_path}")


[STAGE 1] Train only head
Epoch 1/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.2615 - loss: 2.1275 - val_accuracy: 0.1050 - val_loss: 38.1613 - learning_rate: 0.0010
Epoch 2/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.2083 - loss: 2.0406 - val_accuracy: 0.0650 - val_loss: 11.6678 - learning_rate: 0.0010
Epoch 3/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 81s 1s/step - accuracy: 0.2383 - loss: 1.9121 - val_accuracy: 0.1250 - val_loss: 4.8788 - learning_rate: 0.0010
Epoch 4/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.3462 - loss: 1.7455 - val_accuracy: 0.1040 - val_loss: 22.5353 - learning_rate: 0.0010
Epoch 5/5
63/63 ━━━━━━━━━━━━━━━━━━━━ 90s 1s/step - accuracy: 0.3320 - loss: 1.6737 - val_accuracy: 0.0850 - val_loss: 9.2965 - learning_rate: 0.0010
[INFO] Best weights (stage1): outputs_pretrained/20250909_195018_resnet50/best_weights_stage1.keras


#### Interpretazione
 - È normale che lo Stage 1 con backbone congelato dia risultati modesti: la head deve ancora imparare a interpretare le feature di ResNet-50, che sono state addestrate su immagini RGB naturali (gatti, cani, auto…), non su patch multispettrali EuroSAT.
 - Qui stiamo usando solo le 3 bande RGB di EuroSAT, quindi le feature non sono del tutto “slegate” dal dominio ImageNet, ma il modello ha bisogno di Stage 2 (fine-tuning) per adattarsi davvero.

<a id="stage_2_tr"></a>
#### 2) STAGE 2 — fine-tuning: sblocco SOLO l’ultimo stage della ResNet-50

Spesso è meglio partire _scongelando_ solo gli ultimi blocchi del backbone invece di tutta la rete.
Ecco le best practice più usate nel fine-tuning:

Perché non sbloccare tutto subito
 - Catastrophic forgetting: con LR troppo alto o dati pochi, aggiornare tutti i pesi distrugge rapidamente le feature generiche di ImageNet.
 - Overfitting/rumore: più parametri liberi ⇒ più rischio di adattarsi al rumore del tuo dataset.
 - Stabilità: gli strati vicini all’input apprendono pattern “universali” (bordi, texture) che è inutile (o dannoso) ri-allenare.

##### Callback – BACKBONE

In [54]:
early_stop_s2 = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)
reduce_lr_s2 = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.25,
    patience=3,
    min_lr=1e-6
)

In [55]:
# Convenzione semplice: rendiamo addestrabile i layer il cui nome inizia con "conv5_"
for layer in backbone_model.layers:
    if layer.name.startswith("conv5_"):
        layer.trainable = True
    else:
        layer.trainable = False

# Nuovo checkpoint per lo stage 2
ckpt_path_ft = RUN_DIR / "best_weights_stage2.keras"
ckpt_s2 = keras.callbacks.ModelCheckpoint(
    filepath=str(ckpt_path_ft),
    monitor="val_loss", 
    save_best_only=True
)
callbacks_s2 = [early_stop_s2, reduce_lr_s2, ckpt_s2]

In [56]:
# Ricompiliamo con LR più basso (fondamentale nel fine-tuning)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=LR_FT),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
print("\n[STAGE 2] Fine-tuning (unfreeze conv5_*)")
print(f"Backbone trainable layers (should be only conv5_*):",
      sum([int(l.trainable) for l in backbone_model.layers]), "layers")

history_s2 = model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=FT_EPOCHS, 
    callbacks=callbacks_s2, 
    verbose=1
)
print(f"[INFO] Best weights (stage2): {ckpt_path_ft}")


[STAGE 2] Fine-tuning (unfreeze conv5_*)
Backbone trainable layers (should be only conv5_*): 32 layers
Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - accuracy: 0.2035 - loss: 2.2343 - val_accuracy: 0.1430 - val_loss: 2.4775 - learning_rate: 1.0000e-04
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.2335 - loss: 1.9407 - val_accuracy: 0.0700 - val_loss: 2.4235 - learning_rate: 1.0000e-04
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.2370 - loss: 1.8585 - val_accuracy: 0.0790 - val_loss: 2.4146 - learning_rate: 1.0000e-04
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 86s 1s/step - accuracy: 0.2632 - loss: 1.7781 - val_accuracy: 0.0700 - val_loss: 2.4836 - learning_rate: 1.0000e-04
Epoch 5/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.2870 - loss: 1.7073 - val_accuracy: 0.0920 - val_loss: 2.4962 - learning_rate: 1.0000e-04
Epoch 6/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 88s 1s/step - accuracy: 0.3225 - loss: 1.6262 - val_accuracy: 0.0790 - val_loss: 2.5692 

#### 3) Accuratezza

In [59]:
# Dizionario con le metriche registrate (accuracy, loss, val_accuracy, val_loss, ecc.)
metrics = history_s2.history  

# Lista dei valori di accuratezza su validation (uno per epoca)
val_acc_list = metrics["val_accuracy"]

# Migliore accuratezza di validazione raggiunta
best_val_acc = max(val_acc_list)

print(f"[RESULT] Best validation accuracy (Stage 2): {best_val_acc*100:.2f}%")

[RESULT] Best validation accuracy (Stage 2): 14.30%


### Performance del modello

- **Loss (funzione di perdita)**  
  Misura quanto le probabilità stimate dal modello si discostano dalla classe corretta.  
  Per la **sparse categorical crossentropy**:  
  $$
  \text{Loss} = \ell_i = - \frac{1}{N} \sum_{i=1}^{N} \ln \big( p_{\text{classe corretta}, i} \big)
  $$

- **Loss su un batch di $N$ esempi**  
  $$
  \text{Loss}_{\text{batch}} = \frac{1}{N} \sum_{i=1}^{N} \ell_i
  $$

- **Loss complessiva su un’epoca** (media pesata sui batch)  
  $$
  \text{Loss}_{\text{epoch}} = \frac{\sum_{b=1}^{B} N_b \cdot \text{Loss}_{\text{batch}, b}}{\sum_{b=1}^{B} N_b}
  $$

- **Accuracy (accuratezza)**  
  Percentuale di esempi classificati correttamente:  
  $$
  \text{Accuracy} = \frac{\text{numero di previsioni corrette}}{\text{numero totale di esempi}}
  $$

<div style="background:#f3f4f6; color:#1f2937; font-family:Inter, system-ui, -apple-system, Segoe UI, Roboto, Arial, sans-serif; font-size:13px; line-height:1.5; padding:14px 16px; border:1px solid #e5e7eb; border-radius:10px;">
  <h4 style="margin:0 0 8px 0; font-size:14px;">Perché la ResNet-50 ha performato male?</h4>
  <ol style="margin:0 0 10px 18px; padding:0;">
    <li style="margin-bottom:6px;">
      <b>Mismatch di dominio (ImageNet ≠ Sentinel-2)</b><br>
      Pesi pre-addestrati su foto RGB “naturali”, mentre le nostre sono patch satellitari multispettrali. Anche usando solo B4-B3-B2, statistiche e contenuti sono diversi → le feature non si trasferiscono bene.
    </li>
    <li style="margin-bottom:6px;">
      <b>Dataset relativamente piccolo</b><br>
      EuroSAT ≈ 4k patch / 10 classi è modesto per una rete profonda (≈23M parametri) → rischio di overfitting o scarsa generalizzazione.
    </li>
    <li style="margin-bottom:6px;">
      <b>Fine-tuning troppo limitato</b><br>
      È stato sbloccato solo lo stage finale (conv5_*) → l’adattamento potrebbe servire già nei blocchi iniziali; con sblocco parziale la rappresentazione resta “rigida”.
    </li>
    <li style="margin-bottom:6px;">
      <b>Iperparametri non ottimali (LR/scheduler)</b><br>
      LR alto in FT può “rompere” i pesi pre-addestrati; LR troppo basso non adatta. L’esplosione di <i>val_loss</i> in alcune epoche suggerisce LR non ben calibrato.
    </li>
    <li style="margin-bottom:6px;">
      <b>Perdita d’informazione (13 → 3 canali)</b><br>
      Riducendo a RGB si scartano NIR/SWIR ecc., spesso cruciali per distinguere le classi di uso del suolo.
    </li>
    <li style="margin-bottom:6px;">
      <b>Preprocess non perfettamente coerente</b><br>
      Il <code>preprocess_input</code> ImageNet presume distribuzioni “fotografiche”; le patch satellitari hanno statistiche colore/contrasto diverse → prime conv “fuori scala”.
    </li>
  </ol>
  <div style="font-size:12px; color:#374151; background:#eef2ff; border:1px solid #e0e7ff; padding:8px 10px; border-radius:8px;">
    <b>Messaggio:</b> il transfer learning non è universale. Per dati satellitari multispettrali, spesso rendono meglio: CNN leggere “su misura”, adattamenti a 13 bande, o architetture specializzate (es. U-Net; FT più profondo con LR adeguati).
  </div>
(FT: Fine Tuning; LR: Learning Rate; Transfer Learning: è una tecnica in cui si riutilizzano i pesi di un modello già addestrato su un grande dataset come punto di partenza per un nuovo compito; )
</div>

<a id="step6"></a>
## STEP 6 — Confronto con il paper EuroSAT (ResNet-50)

<div style="font-family: 'Helvetica Neue', Arial, sans-serif; font-size:14px; color:#222;">
  <table style="border-collapse:collapse; width:100%; border:1px solid #ddd;">
    <thead>
      <tr style="background:#f0f3f6;">
        <th style="text-align:left; padding:10px; border-bottom:1px solid #ddd; width:16%;">Parametro</th>
        <th style="text-align:left; padding:10px; border-bottom:1px solid #ddd; width:22%;">Implementazione paper</th>
        <th style="text-align:left; padding:10px; border-bottom:1px solid #ddd; width:22%;">Implementazione seminario</th>
        <th style="text-align:left; padding:10px; border-bottom:1px solid #ddd; width:20%;">Confronto</th>
        <th style="text-align:left; padding:10px; border-bottom:1px solid #ddd; width:20%;">Azione migliorativa</th>
      </tr>
    </thead>
    <tbody>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Dataset split</td>
        <td style="padding:10px;">~80% train / 20% test, bilanciato per classe</td>
        <td style="padding:10px;">Uso gli split già generati nel baseline (train/val/test)</td>
        <td style="padding:10px;">Allineati: approccio simile</td>
        <td style="padding:10px;">–</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Bande usate</td>
        <td style="padding:10px;">RGB: B4 (R), B3 (G), B2 (B)</td>
        <td style="padding:10px;">Strategia <code>RGB_only</code> (B4-B3-B2)</td>
        <td style="padding:10px;">Allineati: stesso set di bande</td>
        <td style="padding:10px;">–</td>
      </tr>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Preprocessing immagini</td>
        <td style="padding:10px;">Resize 224×224; mapping 16→8 bit; 0 = nodata</td>
        <td style="padding:10px;">Resize 224×224; uso <code>preprocess_input</code> di ResNet-50; no mapping 16→8</td>
        <td style="padding:10px;">Differenza: manca la conversione 16→8 (può impattare i risultati)</td>
        <td style="padding:10px;">Allineare preprocessing al paper (mapping 16→8) per confronto più corretto</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Modello</td>
        <td style="padding:10px;">ResNet-50 pre-addestrata su ImageNet (1000 classi)</td>
        <td style="padding:10px;">Caricata con <code>weights="imagenet"</code>, <code>include_top=False</code></td>
        <td style="padding:10px;">Allineati</td>
        <td style="padding:10px;">–</td>
      </tr>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Strategia di training</td>
        <td style="padding:10px;">2 fasi:<br>1) Solo testa, LR=0.01<br>2) Fine-tuning intera rete, LR=0.001→0.0001</td>
        <td style="padding:10px;">2 fasi:<br>1) Solo testa, LR=1e-3, epoche=5<br>2) Fine-tuning <i>parziale</i>, LR=1e-4, epoche=15</td>
        <td style="padding:10px;">Differenza: paper fine-tuning totale, seminario solo conv5_*</td>
        <td style="padding:10px;">Provare fine-tuning più esteso (ultimi 2-3 stage) e aumentare epoche</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Epoche</td>
        <td style="padding:10px;">120</td>
        <td style="padding:10px;">Stage1=5, Stage2=15</td>
        <td style="padding:10px;">Paper molto più lungo</td>
        <td style="padding:10px;">Aumentare epoche complessive per stabilizzare il training</td>
      </tr>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Scheduler LR</td>
        <td style="padding:10px;">Riduzione ×0.1 quando val_loss ferma ~5 epoche</td>
        <td style="padding:10px;"><code>ReduceLROnPlateau</code> (factor=0.5, patience=2, min_lr=1e-6)</td>
        <td style="padding:10px;">Simili, parametri diversi</td>
        <td style="padding:10px;">Testare decay più aggressivo (×0.1) come nel paper</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Data augmentation</td>
        <td style="padding:10px;">Flip H, shearing (0.2), zoom (0.2)</td>
        <td style="padding:10px;">Solo flip orizzontale</td>
        <td style="padding:10px;">Differenza: augment ridotta</td>
        <td style="padding:10px;">Integrare shear e zoom per ridurre overfitting</td>
      </tr>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Loss</td>
        <td style="padding:10px;">Categorical crossentropy</td>
        <td style="padding:10px;">Sparse categorical crossentropy</td>
        <td style="padding:10px;">Equivalenti: formato etichette diverso</td>
        <td style="padding:10px;">–</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Optimizer</td>
        <td style="padding:10px;">SGD<sup>§</sup> (Stochastic Gradient Descent)</td>
        <td style="padding:10px;">Adam<sup>†</sup></td>
        <td style="padding:10px;">Differenza: approccio diverso</td>
        <td style="padding:10px;">Provare SGD con momentum per allinearsi al paper</td>
      </tr>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Momentum</td>
        <td style="padding:10px;">0.9 (tipico)</td>
        <td style="padding:10px;">–</td>
        <td style="padding:10px;">Paper lo specifica, seminario non usato</td>
        <td style="padding:10px;">Introdurre momentum se si usa SGD</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Batch size</td>
        <td style="padding:10px;">16</td>
        <td style="padding:10px;">64</td>
        <td style="padding:10px;">Differenza significativa</td>
        <td style="padding:10px;">Ridurre batch size per avvicinarsi al setup del paper</td>
      </tr>
      <tr style="background:#ffffff;">
        <td style="padding:10px;">Callback</td>
        <td style="padding:10px;">LR scheduler; early stopping opzionale</td>
        <td style="padding:10px;">EarlyStopping, ReduceLROnPlateau, ModelCheckpoint</td>
        <td style="padding:10px;">Allineati</td>
        <td style="padding:10px;">–</td>
      </tr>
      <tr style="background:#fafafa;">
        <td style="padding:10px;">Accuratezza riportata</td>
        <td style="padding:10px;"><b>~98.6%</b> su EuroSAT (RGB)</td>
        <td style="padding:10px;">14.50%</td>
        <td style="padding:10px;">Differenza significativa</td>
        <td style="padding:10px;">Uniformare setup (preprocessing, optimizer, augment) per avvicinarsi</td>
      </tr>
    </tbody>
  </table>

  <p style="font-size:12px; color:#555; margin-top:10px;">
    <b><sup>§</sup></b> SGD aggiorna i pesi seguendo il gradiente medio; con momentum (tipico 0.9) accumula velocità e stabilizza la discesa. <br>
    <b><sup>†</sup></b> Adam adatta il learning rate parametro per parametro stimando media e varianza dei gradienti, più rapido a convergere ma può generalizzare peggio.
  </p>
</div>